In [1]:
# Version notes
# V4: added highway and transit perceived travel time accessibility measures, added percentage of regional employment accessible per employment category
# V5: correct errors with transit access not being available
# V6: add drive-access transit, combine with walk access; change access to EmpRes to reflect VOT/Income at destination

import pandas as pd
import numpy as np
from collections import defaultdict
from openpyxl import Workbook
import openmatrix as omx
import glob, os, sys
from datetime import datetime

In [2]:
#skim_dir = r'C:\MTC_tmpy\TM2\tm2py\examples\Link21_3332\skims\accessibility'
skim_dir = r'C:\Users\jgliebe\Documents\Projects\BART\Accessibilities\variables\acc_skims'
os.chdir(skim_dir)

current_date = datetime.now().strftime('%Y%m%d')
#out_excel_fn = r'C:\MTC_tmpy\TM2\tm2py\examples\Link21_3332\skims\accessibility\acc_measures_{date}.xlsx'.format(date = current_date)
out_excel_fn = r'C:\Users\jgliebe\Documents\Projects\BART\Accessibilities\variables\accessibilityacc_measures_{date}.xlsx'.format(date = current_date)
writer = pd.ExcelWriter(out_excel_fn, engine = 'openpyxl')
writer.book = Workbook()

#df_land_use = pd.read_csv(r"C:\MTC_tmpy\TM2\tm2py\examples\Link21_3332\inputs\landuse\tazData.csv")
df_land_use = pd.read_csv(r"C:\Users\jgliebe\Documents\Projects\BART\Accessibilities\variables\tazData.csv")
num_zones = len(df_land_use)
tt_matrices = {}

#TODs = ['EA','AM','MD','PM','EV']
TODs = ['AM'] #Can change later to include all TODs

emp_type = ['TOTEMP','RETEMPN','HEREMPN','EMPRES']
cutoff_start = [0, 10, 20, 30, 40, 50, 60, 70]
cutoff_end = [10, 20, 30, 40, 50, 60, 70, 80]

perc_trans_offset = 40  # offset transit cutoff ranges due to perceived walk transit times being so much longer
factor_trans_binsize = 1.5

#### Simple

In [3]:
acc_mode_groups = {'highway':{'mode':'highway','omx_name':'HWYSKM','core':'TIMEDA'},
                   'walktrans':{'mode':'wtransit','omx_name':'WLK_TRN','core':'TRN_TOT_TIME'},
                   'drivetrans':{'mode':'dtransit','omx_name':'PNR_TRN','core':'TRN_TOT_TIME'},
                   'alltrans':{'mode':'atransit','omx_name':'','core':''},
                   'nonmotor':{'mode':'non-motorized','omx_name':'nonmotskm','core':'DISTWALK'}}

In [4]:
# Aggregate transit travel times into a single TRN_TOT_TIME core, divided by 100
for tod in TODs:
    for fn in glob.glob(f'*trnskm*.omx'):
        if tod.lower() in fn.lower():
            with omx.open_file(fn, 'a') as f:
                if 'IVTX' in f.list_matrices():
                    del f['IVTX']
                if 'TRN_TOT_TIME' in f.list_matrices():
                    del f['TRN_TOT_TIME']
                ivt = np.array(f['IVT'])
                ivt1 = ivt * (ivt>0)
                ivt0 = 999999 * (ivt==0)
                ivtx = ivt0 + ivt1
                f['IVTX'] = ivtx.reshape(len(f['IVT']),len(f['IVT']))
                f['TRN_TOT_TIME'] = np.add.reduce([np.array(f[mat]) \
                                                   for mat in ['DTIME','IVTX','IWAIT','XWAIT','WACC','WAUX','WEGR']])/ 100
#                 print(f.list_matrices())

In [5]:
# Gather processed skim matrices
for access_type in acc_mode_groups:
    print(access_type)
    if access_type != 'alltrans':
        for tod in TODs:
            fname = acc_mode_groups[access_type]['omx_name']
            for fn in glob.glob(f'*{fname}*.omx'):
                if access_type == 'nonmotor' or tod.lower() in fn.lower():
                    with omx.open_file(fn) as f:
                        mode = acc_mode_groups[access_type]['mode']
                        skim_array = np.array(f[acc_mode_groups[access_type]['core']])
                        if acc_mode_groups[access_type]['core'] =='DISTWALK':
                            skim_array = skim_array*20
                        tt_matrices[(tod,mode)] = skim_array[:num_zones, :num_zones]
    else:
        pass

highway
walktrans
drivetrans
alltrans
nonmotor


In [6]:
# Choose the better travel time between walk-access transit and drive-access transit
for tod in TODs:
    fnames = [f for f in glob.glob(f'*trnskm*.omx') if tod.lower() in f.lower()]
    ft = omx.open_file(fn, 'a')
    for fn in fnames:
        if acc_mode_groups['walktrans']['omx_name'] in fn:
            fw = omx.open_file(fn, 'a')
            if 'USE' in fw.list_matrices():
                del fw['USE']
        elif acc_mode_groups['drivetrans']['omx_name'] in fn:
            fd = omx.open_file(fn, 'a')
            if 'USE' in fd.list_matrices():
                del fd['USE']
        else:
            break
    # Compare total transit travel times
    watt = np.array(fw['TRN_TOT_TIME'])
    datt = np.array(fd['TRN_TOT_TIME'])
    w1 = (watt <= datt) * 1
    d1 = (watt > datt) * 1
    # Create indicator cores
    fw['USE'] = w1 = w1.reshape(len(fw['IVT']),len(fw['IVT']))
    fd['USE'] = d1 = d1.reshape(len(fd['IVT']),len(fd['IVT']))
    fw.close()
    fd.close()
    
    # Create a blend of the best transit travel times
    tt_matrices[(tod,'atransit')] = tt_matrices[(tod,'wtransit')] * w1 + tt_matrices[(tod,'dtransit')] * d1

In [7]:
all_zones_df = pd.DataFrame(0, index = df_land_use.ZONE.values, columns = [])
all_zones_df.index.name = 'zone_ID'

all_zones_pct_df = pd.DataFrame(0, index = df_land_use.ZONE.values, columns = [])
all_zones_pct_df.index.name = 'zone_ID'

In [8]:
for employment in emp_type:
    for tod in TODs:
        for access_type in acc_mode_groups:
            mode = acc_mode_groups[access_type]['mode']
            new_key = (tod,mode)
            if new_key in tt_matrices and mode not in ('wtransit','dtransit'):
                print(employment, new_key)
                for (cutoff_s, cutoff_e) in zip(cutoff_start, cutoff_end):
                    if mode == 'dtransit':
                        cutoff_s *= factor_dtrans_binsize
                        cutoff_e *= factor_dtrans_binsize
                        if cutoff_s > 0: 
                            cutoff_s += perc_dtrans_offset
                            cutoff_e += perc_dtrans_offset
                        else:
                            cutoff_e += perc_dtrans_offset                      
                    grp_tt = (tt_matrices[(tod,mode)])
                    grp_tt = ((grp_tt >= cutoff_s) & (grp_tt < cutoff_e)).astype(int)
                    all_zones_df[f'{employment}_{tod}_{mode}_{int(cutoff_s)}_{int(cutoff_e)}'] = (grp_tt*df_land_use[f'{employment}'].values).sum(axis=1)
                    all_zones_pct_df[f'{employment}_{tod}_{mode}_{int(cutoff_s)}_{int(cutoff_e)}'] = (grp_tt*df_land_use[f'{employment}'].values).sum(axis=1) / df_land_use[f'{employment}'].sum()

TOTEMP ('AM', 'highway')
TOTEMP ('AM', 'atransit')
TOTEMP ('AM', 'non-motorized')
RETEMPN ('AM', 'highway')
RETEMPN ('AM', 'atransit')
RETEMPN ('AM', 'non-motorized')
HEREMPN ('AM', 'highway')
HEREMPN ('AM', 'atransit')
HEREMPN ('AM', 'non-motorized')
EMPRES ('AM', 'highway')
EMPRES ('AM', 'atransit')
EMPRES ('AM', 'non-motorized')


In [9]:
all_zones_df.to_excel(writer,sheet_name = 'simple')
all_zones_pct_df.to_excel(writer,sheet_name = 'simple_PCT')

writer.save()

#### Perceived

In [10]:
# Accessibility based on perceived time/cost
ptt0_matrices = {} # origin-based income/vot for household accessibility to attractions
ptt1_matrices = {} # destination-based income/vot for establishment accessibility to workers

In [11]:
# Highway parameters
segment_suffixes = ["LowInc", "MedInc", "HighInc", "XHighInc"]
cutoffs = [0, 30000, 60000, 100000]
VOTs = {"LowInc": 6.01, "MedInc": 8.81, "HighInc": 10.44, "XHighInc": 12.86} # uses VOT according to mean HH income per TAZ
hh = pd.read_csv('hhFile.2015.csv')
hh_mean_inc = hh.groupby(['TAZ'])['HINC'].mean().reindex(df_land_use.ZONE.values).rename('mean_inc')
hh_mean_inc = hh_mean_inc.fillna(hh.HINC.mean())
assert len(hh_mean_inc) == len(df_land_use) and hh_mean_inc.isna().sum() == 0, 'HH mean income is incomplete'
hh_mean_inc = hh_mean_inc.reset_index()
hh_mean_inc['income_seg'] = pd.cut(hh_mean_inc['mean_inc'], right = False, bins = cutoffs + [float('inf')], labels = segment_suffixes).astype(str)
hh_mean_inc['VOT_per_hour'] = hh_mean_inc['income_seg'].map(VOTs) 
hh_mean_inc['VOT_per_min'] = hh_mean_inc['VOT_per_hour']/ 60 # values from VOTs are in $/hour, convert into $/minute

In [12]:
# Cost units are cents, expressed in year 2000 dollars.
for tod in TODs:
    for fn in glob.glob('hwyskm*.omx'):
        if tod in fn:
            with omx.open_file(fn) as f:
                mode = 'highway'
                time_array = np.array(f['TIMEDA'])[:num_zones, :num_zones]
                cost_array = np.array(f['COSTDA'])[:num_zones, :num_zones] / 100
                toll_array = np.array(f['BTOLLDA'])[:num_zones, :num_zones] / 100 \
                + np.array(f['VTOLLDA'])[:num_zones, :num_zones] / 100
                
                # broadcasting VOT values horizontally to get VOT by origin zone
                p0_time_array = time_array \
                + np.divide(cost_array, hh_mean_inc.VOT_per_min.values[:,None]) \
                + np.divide(toll_array, hh_mean_inc.VOT_per_min.values[:,None])  
                ptt0_matrices[(tod, mode)] = p0_time_array
                
                # broadcasting VOT values vertically to get VOT by destination zone
                p1_time_array = time_array \
                + np.divide(cost_array, hh_mean_inc.VOT_per_min.values[None,:]) \
                + np.divide(toll_array, hh_mean_inc.VOT_per_min.values[None,:])  
                ptt1_matrices[(tod, mode)] = p1_time_array

In [13]:
# Transit
waitThresh = 10 # 10 minutes per UEC
coef_dict = {'IWAIT_S': 2, 'IWAIT_L': 1, 'XWAIT' : 2, 'WACC' : 2, \
             'WEGR' : 2, 'WAUX' : 2, 'CROWD' : 1.62, 'DTIME': 2} # 1.62

#c_shortiWait	Short initial wait time coefficient -- see "waitThresh"	2.00 * c_ivt
#c_longiWait	Long initial wait time coefficient -- see "waitThresh"	1.00 * c_ivt
#c_wacc	Walk access time coefficient	2.00 * c_ivt
#c_wegr	Walk egress time coefficient	2.00 * c_ivt
#c_xwait Transfer wait time coefficient	2.00 * c_ivt
#c_waux	Walk auxilliary time coefficient		2.00 * c_ivt

# short wait time: c_shortiWait*min(WLK_TRN_WLK_IWAIT[tripPeriod]/100,waitThresh)
# long wait time: c_longiWait*max(WLK_TRN_WLK_IWAIT[tripPeriod]/100-waitThresh,0)
# is the initial wait time reflected as a sum of both?

In [14]:
for tod in TODs:
    for fn in glob.glob('*trnskm*.omx'):
        if tod.lower() in fn.lower():
            with omx.open_file(fn) as f:
                ptt = [np.array(f['IVTX']) / 100]
                fare_array = np.array(f['FARE'])[:num_zones, :num_zones] / 100
                if 'PNR_TRN' in fn:
                    mode = 'dtransit'
                if 'WLK_TRN' in fn:
                    mode = 'wtransit'
                for OVTT in ['XWAIT','WACC','WEGR','WAUX','CROWD','DTIME']:
                    ptt.append(np.array(f[OVTT]) * coef_dict[OVTT] / 100)
                IWAIT = coef_dict['IWAIT_S'] * np.array(f['IWAIT']) / 100 \
                + coef_dict['IWAIT_L'] * np.clip(np.array(f['IWAIT'])/100 - waitThresh, a_min = 0, a_max = None)
                ptt.append(IWAIT)
                
                # broadcasting VOT values horizontally to get VOT by origin zone
                ptt0_matrices[(tod, mode)] = np.add.reduce([mat[:num_zones, :num_zones] for mat in ptt]) \
                + np.divide(fare_array, hh_mean_inc.VOT_per_min.values[:,None])
                
                # broadcasting VOT values vertically to get VOT by destination zone for access to workers
                ptt1_matrices[(tod, mode)] = np.add.reduce([mat[:num_zones, :num_zones] for mat in ptt]) \
                + np.divide(fare_array, hh_mean_inc.VOT_per_min.values[None,:])
        
    # Create a blend of the best transit travel times
    w2 = (ptt0_matrices[(tod,'wtransit')] <= ptt0_matrices[(tod,'dtransit')]) * 1
    d2 = (ptt0_matrices[(tod,'wtransit')] > ptt0_matrices[(tod,'dtransit')]) * 1
    
    w3 = (ptt1_matrices[(tod,'wtransit')] <= ptt1_matrices[(tod,'dtransit')]) * 1
    d3 = (ptt1_matrices[(tod,'wtransit')] > ptt1_matrices[(tod,'dtransit')]) * 1
    
    ptt0_matrices[(tod,'atransit')] = ptt0_matrices[(tod,'wtransit')] * w2 + ptt0_matrices[(tod,'dtransit')] * d2
    ptt1_matrices[(tod,'atransit')] = ptt1_matrices[(tod,'wtransit')] * w3 + ptt1_matrices[(tod,'dtransit')] * d3

In [15]:
test1 = ((w2!=w1)*1).sum()/(len(w2)*len(w1))*100
print("%6.3f pct." % test1)
test2 = ((w2!=w3)*1).sum()/(len(w2)*len(w3))*100
print("%6.3f pct." % test2)
test3 = ((w1!=w3)*1).sum()/(len(w1)*len(w3))*100
print("%6.3f pct." % test3)

 5.951 pct.
 1.021 pct.
 6.033 pct.


In [16]:
all_zones_df = pd.DataFrame(0, index = df_land_use.ZONE.values, columns = [])
all_zones_df.index.name = 'zone_ID'

all_zones_pct_df = pd.DataFrame(0, index = df_land_use.ZONE.values, columns = [])
all_zones_pct_df.index.name = 'zone_ID'

In [17]:
for employment in emp_type:
    if employment == 'EMPRES':
        ptt_matrices = ptt1_matrices
    else: 
        ptt_matrices = ptt0_matrices
    for tod in TODs:        
        for mode in ['highway','atransit']:
            new_key = (tod,mode)
            if new_key in ptt_matrices:
                for (cutoff_s, cutoff_e) in zip(cutoff_start, cutoff_end):
                    if mode == 'atransit':
                        cutoff_s *= factor_trans_binsize
                        cutoff_e *= factor_trans_binsize
                        if cutoff_s > 0: 
                            cutoff_s += perc_trans_offset
                            cutoff_e += perc_trans_offset
                        else:
                            cutoff_e += perc_trans_offset                              
                    grp_tt = (ptt_matrices[(tod,mode)])
                    grp_tt = ((grp_tt >= cutoff_s) & (grp_tt < cutoff_e)).astype(int)

                    all_zones_df[f'{employment}_{tod}_{mode}_{int(cutoff_s)}_{int(cutoff_e)}'] = (grp_tt*df_land_use[f'{employment}'].values).sum(axis=1)
                    all_zones_pct_df[f'{employment}_{tod}_{mode}_{int(cutoff_s)}_{int(cutoff_e)}'] = (grp_tt*df_land_use[f'{employment}'].values).sum(axis=1) / df_land_use[f'{employment}'].sum()

In [18]:
all_zones_df.to_excel(writer,sheet_name = 'perceived_TT')
all_zones_pct_df.to_excel(writer,sheet_name =  'perceived_TT_PCT')
hh_mean_inc[['TAZ','mean_inc', 'income_seg', 'VOT_per_hour', 'VOT_per_min']].to_excel(writer,sheet_name = 'HH inc & VOT', index = False)

writer.save()